In [2]:
%load_ext watermark
%watermark -a Chan -d -v -p numpy,pandas,konlpy,tensorflow

Chan 2019-08-13 

CPython 3.7.3
IPython 7.6.1

numpy 1.16.4
pandas 0.24.2
konlpy 0.5.1
tensorflow 1.13.1


In [3]:
import numpy as np
import pandas as pd
from konlpy.tag import Okt
import tensorflow as tf
from tensorflow.python.keras.preprocessing import sequence
from tensorflow import keras
from tqdm import tqdm
from collections import Counter
import pickle

In [4]:
# 각 동영상별로 count해야한다. 일단 dump파일로 짜놓기

In [5]:
datasets = pd.DataFrame()
for i in range(9,10):
    print(i)
    dump = pd.read_csv("../../data/labeled ({}).csv".format(i), engine='python')
    datasets = pd.concat([datasets, dump])
datasets.head()

9


,comment,url_id,label
0,궁,23,0
1,진에어 화이팅!,11,0
2,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,12,0
3,싹둑이 채팅창 관리 중입니다,34,0
4,앗,52,0


In [8]:
datasets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 3 columns):
comment    3000 non-null object
url_id     3000 non-null int64
label      3000 non-null int64
dtypes: int64(2), object(1)
memory usage: 70.4+ KB


In [9]:
datasets.fillna('null',inplace=True)

In [10]:
len(datasets)

3000

In [11]:
morp = Okt()
morped = [morp.morphs(_, norm=True, stem=True) for _ in tqdm(datasets.comment)]

C:\Users\skarn\Anaconda3\envs\DataAnalysis\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)
100%|█████████████████████████████████████████████████████████████████████████████| 3000/3000 [00:26<00:00, 114.84it/s]


In [12]:
morped[:5]

[['궁'],
 ['진에어', '화이팅', '!'],
 ['ㅋㅋㅋ'],
 ['싹둑', '이', '채팅', '창', '관리', '중', '이다'],
 ['앗']]

In [13]:
# load vocab
with open('./vocab/vocab_index.pickle', 'rb') as f:
    vocab_index = pickle.load(f)
with open('./vocab/vocab_inverted_index.pickle', 'rb') as f:
    vocab_inverted_index = pickle.load(f)

In [14]:
def text_to_index(tokens):
    oov_id = 1
    indexes = []
    for tok in tokens:
        if tok in vocab_index:
            indexes.append(vocab_index[tok])
        else:
            indexes.append(oov_id)

    return indexes

In [15]:
pad_id = 0

x_variable = [text_to_index(_) for _ in morped]

sentence_size = 10
x_padded = sequence.pad_sequences(x_variable,
                                 maxlen=sentence_size,
                                 truncating='post',
                                 padding='post',
                                 value=pad_id)

# Load Model 

In [16]:
up_model = keras.models.load_model('../model/cnn_oversample.h5')
down_model =keras.models.load_model('../model/cnn_undersample.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [17]:
datasets['predict_up'] = up_model.predict_classes(x_padded)
datasets['predict_down'] = down_model.predict_classes(x_padded)

In [18]:
up_model.evaluate(x_padded, pd.get_dummies(datasets.label))[1], down_model.evaluate(x_padded, pd.get_dummies(datasets.label))[1] # loss, acc

3000/3000 [==============================] - ETA: 48s - loss: 0.5622 - recall_1: 0.875 - ETA: 2s - loss: 0.7663 - recall_1: 0.731 - ETA: 1s - loss: 0.7285 - recall_1: 0.73 - ETA: 0s - loss: 0.7072 - recall_1: 0.73 - ETA: 0s - loss: 0.7305 - recall_1: 0.72 - ETA: 0s - loss: 0.7454 - recall_1: 0.72 - 1s 277us/sample - loss: 0.7388 - recall_1: 0.7250


(0.9073333, 0.725)

## 비속어 사전 기반 필터링의 경우

In [19]:
from auto_label import auto_label
import pandas as pd

In [20]:
result = auto_label(datasets,'./word-fobidden.json')

In [21]:
result.head()

,comment,url_id,label,predict_up,predict_down,filtered
0,궁,23,0,0,0,0
1,진에어 화이팅!,11,0,0,0,0
2,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,12,0,0,0,0
3,싹둑이 채팅창 관리 중입니다,34,0,0,0,0
4,앗,52,0,0,0,0


In [22]:
recall_data = result[result.label==1]

In [32]:
sum(recall_data.predict_up)/len(recall_data), \
# sum(recall_data.predict_down)/len(recall_data) 

# 모델 방법의 recall

(0.4970414201183432,)

In [24]:
sum(recall_data.filtered)/len(recall_data) # 비속어사전 방법의 recall

0.17751479289940827

In [27]:
sum(datasets.filtered==datasets.label) /len(datasets)# 비속어사전 방법의 accuracy

0.9443333333333334

#### 전체 채팅 중 유해채팅 비율

In [31]:
datasets.predict_up.sum() / len(datasets)

0.09233333333333334

> 8%가 적당하다

In [28]:
recall_data.to_csv('./result/cnn_result.csv', encoding='euc-kr')